# Imports and change directory

In [176]:
import os
import folium
import pandas as pd
import geopandas as gpd

P_DIR="/Users/Licious/Downloads/Project Data-20221102/2109_STIB_MIVB_Network"
os.chdir(P_DIR)


All files in stib


In [177]:
files_path =  os.listdir()
print(files_path)

['svg', 'ACTU_STOPS.cpg', 'ACTU_STOPS.shp', 'test', 'ACTU_STOPS.shx', 'ACTU_STOPS.sbx', 'ACTU_LINES.prj', 'plugins', 'ACTU_LINES.sbn', 'ACTU_STOPS.dbf', 'ACTU_LINES.shp.xml', 'ACTU_STOPS.prj', 'sample.json', 'ACTU_LINES.dbf', 'ACTU_STOPS.sbn', 'ACTU_LINES.shp', 'ACTU_LINES.cpg', 'ACTU_LINES.shx', 'fonts', 'ACTU_STOPS.shp.xml', 'ACTU_LINES.sbx', 'conf', 'a.html']


# Read all files

In [225]:
stop_gdf = gpd.read_file('ACTU_STOPS.shp')
lines_gdf = gpd.read_file('ACTU_LINES.shp')
stop_col = ["Code_Ligne","Variante","succession","stop_id","descr_fr","descr_nl","alpha_fr","alpha_nl","coord_x","coord_y","mode","numero_lig","terminus"]
lines_gdf.head()


,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,geometry
0,001m,1,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (146633.500 170956.400 0.000, 146..."
1,001m,2,#C4008F,01/09/2021,06/03/2022,"LINESTRING Z (156746.700 170167.000 0.000, 156..."
2,002m,1,#F57000,01/09/2021,06/03/2022,"LINESTRING Z (147305.500 172526.900 0.000, 147..."
3,002m,2,#F57000,01/09/2021,06/03/2022,"LINESTRING Z (147370.500 172498.500 0.000, 147..."
4,003t,1,#B5BA05,01/09/2021,06/03/2022,"LINESTRING Z (148550.000 176641.300 0.000, 148..."


In [234]:
test_data=lines_gdf.loc[lines_gdf["LIGNE"]=="093t"]
test_data.head()
test_data.loc[test_data['VARIANTE'] == 1, 'COLOR_HEX'] = 'purple'
test_data.loc[test_data['VARIANTE'] == 2, 'COLOR_HEX'] = 'red'

test_data.head()

,LIGNE,VARIANTE,COLOR_HEX,Date_debut,Date_fin,geometry
144,093t,1,purple,01/09/2021,06/03/2022,"LINESTRING Z (147422.300 175898.100 0.000, 147..."
145,093t,2,red,01/09/2021,06/03/2022,"LINESTRING Z (150206.600 167361.000 0.000, 150..."


##  Transformer code below need to check properly. Can be used in future

#   Mapping betweeen tram and mode of transport

In [179]:
mode_line_mapping = {
    "T": [7, 93, 81, 51, 19, 9, 39, 97, 82, 32, 44, 25, 92, 8, 3, 55, 4, 62],
    "B": [
        98,
        87,
        77,
        61,
        86,
        213,
        206,
        38,
        49,
        204,
        12,
        70,
        33,
        56,
        212,
        46,
        53,
        59,
        80,
        89,
        209,
        76,
        60,
        66,
        27,
        48,
        57,
        83,
        47,
        78,
        41,
        14,
        216,
        17,
        64,
        42,
        210,
        21,
        88,
        75,
        36,
        13,
        54,
        50,
        52,
        63,
        95,
        69,
        72,
        34,
        65,
        205,
        28,
        29,
        43,
        71,
        58,
        79,
        208,
        45,
        218,
        74,
        37,
        20,
        211,
    ],
    "M": [5, 1, 6, 2],
}


# Map global code

In [241]:
# Map start
map = folium.Map(location=[50.8260,4.3802],zoom_start=15)


# Below code will handle all manipulation related to stops

In [242]:
# modes = ['M']
#Below is  the sample
#select_lines = {'M':'*','T':[93, 81, 51]}

select_lines = {'T':[93]}
color_select = {"T": "black", "B": "blue", "M":"yellow"}
icon_select = {"T": "plane", "B": "bus", "M":"dribbble"}
selected_lines = []
for mode_chosen in select_lines.keys():
    filtered_data = stop_gdf.loc[stop_gdf['mode']==mode_chosen]
    if select_lines[mode_chosen]!='*':
        filtered_data = filtered_data.loc[filtered_data['numero_lig'].isin(select_lines[mode_chosen])]
    selected_lines = selected_lines+list(set(filtered_data['Code_Ligne']))
    folium.GeoJson(data=filtered_data,
        popup=folium.GeoJsonPopup(stop_col),
        marker = folium.Marker(icon=folium.Icon(icon=icon_select[mode_chosen], prefix='fa' , color=color_select[mode_chosen], icon_color='white'
    ))).add_to(map)



In [221]:
print(selected_lines)

['093t']


In [182]:
lines_gdf.to_html("a.html")

In [183]:
! open a.html

16397.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [243]:
style1 = {'fillColor': '#228B22', 'lineColor': 'white'}
# selected_lines = select_lines.map()
folium.GeoJson(
    data=test_data
    ,popup=folium.GeoJsonPopup(["LIGNE",]),
    style_function= lambda x: {
    'color' : x['properties']['COLOR_HEX'],
    'weight' : 4 if x['properties']['LIGNE'][-1]=='m' else 4,
    'opacity' : 1 if x['properties']['LIGNE'] in selected_lines else 0
}
    # marker = folium.Marker(icon=folium.Icon(icon=icon_select[mode_chosen], prefix='fa' , color=color_select[mode_chosen], icon_color='white'
    # )
    # )
    ).add_to(map)

In [244]:
map